In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.metrics import recall_score, accuracy_score
from sklearn.impute import KNNImputer, SimpleImputer

In [2]:
random_state = 7
cv_method = StratifiedKFold(n_splits=5)

# Training of tsfresh-selected Training Max 1 years and eGFR 4 times

In [3]:
df = pd.read_csv('/home/jupyter-dchristiadi85/PhD Project 1/data/train_selected_1half_6.gz')
df.shape

(4864, 24364)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4864 entries, 0 to 4863
Columns: 24364 entries, id to cat8
dtypes: float64(24342), int64(2), object(20)
memory usage: 904.1+ MB


In [5]:
df['aki_1'].fillna(value=0, inplace=True)
df['aki_2'].fillna(value=0, inplace=True)
df['aki_3'].fillna(value=0, inplace=True)

In [6]:
df['gn'].value_counts()

no                 4420
igan                180
anca                 82
lupus nephritis      45
membranous           35
unspecific           29
fsgs                 27
mcd                  22
mp/mcgn              16
fgn/itg               6
pign                  2
Name: gn, dtype: int64

In [7]:
df.loc[df.gn == 'pign', 'gn'] = 'unspecific'
df.loc[df.gn == 'fgn/itg', 'gn'] = 'unspecific'

In [8]:
percent_missing = df.isnull().sum()*100 / len(df)
missing_df = pd.DataFrame({'column_name': df.columns, 'percent_miss': percent_missing})
missing_df.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
caphos_product__query_similarity_count__query_None__threshold_0.0,caphos_product__query_similarity_count__query_...,100.0
"globulin__fft_coefficient__attr_""""""""""""""""angle""""""""""""""""__coeff_96","globulin__fft_coefficient__attr_""""""""""""""""angle""...",100.0
"alt__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""__coeff_95","alt__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""...",100.0
"alt__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""__coeff_94","alt__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""...",100.0
"alt__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""__coeff_93","alt__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""...",100.0
...,...,...
htn,htn,0.0
aki_3,aki_3,0.0
aki_2,aki_2,0.0
aki_1,aki_1,0.0


### Number of patients reaching ESKD within training period 

In [9]:
df['cat1.5'].value_counts()

non_eskd    3675
eskd        1189
Name: cat1.5, dtype: int64

In [10]:
df.set_index('id', inplace=True)

In [11]:
def to_category(df):
    cols = df.select_dtypes(include='object').columns
    for col in cols:
        df[col] = df[col].astype('category')
    return df

def drop_missing(df):
    threshold = len(df)*0.6
    df.dropna(axis=1, thresh=threshold, inplace=True)
    return df

def copy_df(df):
    return df.copy()

In [12]:
df_cleaned = (df.pipe(copy_df).pipe(drop_missing).pipe(to_category))
df_cleaned.shape

(4864, 6310)

In [30]:
df_cleaned.columns[-20:]

Index(['gender', 'value_egfr_init', 'egfr.y', 'age.init', 'cat0.5', 'cat1',
       'eskd_filter', 'cat2', 'cat2.5', 'cat3', 'cat3.5', 'cat4', 'cat4.5',
       'cat5', 'cat5.5', 'cat6', 'cat6.5', 'cat7', 'cat7.5', 'cat8'],
      dtype='object')

In [32]:
df_rm_intrain = df_cleaned.query("eskd_filter == 'non_eskd'")
df_rm_intrain.shape

(3675, 6310)

In [6]:
df_rm_intrain = pd.read_csv('../data/cleaned60_selected_1half_6.gz', index_col='id')

In [33]:
df_rm_intrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3675 entries, 3916 to 23864
Columns: 6310 entries, urea__variance_larger_than_standard_deviation to cat8
dtypes: category(20), float64(6289), int64(1)
memory usage: 176.5 MB


In [8]:
df_rm_intrain = (df_rm_intrain.pipe(copy_df).pipe(to_category))
df_rm_intrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4409 entries, 3916 to 23864
Columns: 5361 entries, mchc__variance_larger_than_standard_deviation to cat8
dtypes: category(20), float64(5340), int64(1)
memory usage: 179.8 MB


### Number of patients reaching ESKD in at 1y post-training 

In [34]:
df_rm_intrain['cat2.5'].value_counts()

non_eskd    3554
eskd         121
Name: cat2.5, dtype: int64

### Number of patients reaching ESKD in 2y post-training

In [35]:
df_rm_intrain['cat3.5'].value_counts()

non_eskd    3444
eskd         231
Name: cat3.5, dtype: int64

### Number of patients reaching ESKD in 5y post-training

In [36]:
df_rm_intrain['cat6.5'].value_counts()

non_eskd    3200
eskd         475
Name: cat6.5, dtype: int64

In [37]:
percent_missing = df_rm_intrain.isnull().sum()*100 / len(df_rm_intrain)
missing_df_rm_intrain = pd.DataFrame({'column_name': df_rm_intrain.columns, 'percent_miss': percent_missing})
missing_df_rm_intrain.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
creatinine__partial_autocorrelation__lag_1,creatinine__partial_autocorrelation__lag_1,51.129252
creatinine__autocorrelation__lag_3,creatinine__autocorrelation__lag_3,51.129252
"creatinine__augmented_dickey_fuller__attr_""""""""""""""""teststat""""""""""""""""__autolag_""""""""""""""""AIC""""""""""""""""","creatinine__augmented_dickey_fuller__attr_""""""""...",51.129252
"creatinine__augmented_dickey_fuller__attr_""""""""""""""""pvalue""""""""""""""""__autolag_""""""""""""""""AIC""""""""""""""""","creatinine__augmented_dickey_fuller__attr_""""""""...",51.129252
"creatinine__fft_coefficient__attr_""""""""""""""""imag""""""""""""""""__coeff_2","creatinine__fft_coefficient__attr_""""""""""""""""imag...",51.074830
...,...,...
dkd,dkd,0.000000
htn,htn,0.000000
aki_3,aki_3,0.000000
aki_1,aki_1,0.000000


## ESKD Prediction 5 years post-training

In [40]:
dropped_cols = list(df_rm_intrain.columns[-18:])
dropped_cols.remove('age.init')
dropped_cols

['egfr.y',
 'cat0.5',
 'cat1',
 'eskd_filter',
 'cat2',
 'cat2.5',
 'cat3',
 'cat3.5',
 'cat4',
 'cat4.5',
 'cat5',
 'cat5.5',
 'cat6',
 'cat6.5',
 'cat7',
 'cat7.5',
 'cat8']

In [41]:
X = df_rm_intrain.drop(dropped_cols,axis=1).copy()
y = df_rm_intrain['cat6.5']
X.shape, y.shape

((3675, 6293), (3675,))

In [42]:
y_mapped = y.map({'non_eskd':'no', 'eskd':'yes'})
y_mapped

id
3916     no
3918     no
3921     no
3924     no
3930     no
         ..
23760    no
23771    no
23780    no
23823    no
23864    no
Name: cat6.5, Length: 3675, dtype: category
Categories (2, object): ['yes', 'no']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, stratify=y_mapped, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2572, 6293), (1103, 6293), (2572,), (1103,))

In [44]:
lab_encode = LabelEncoder()
y_train = lab_encode.fit_transform(y_train)
y_test = lab_encode.transform(y_test)
y_train.shape, y_test.shape

((2572,), (1103,))

In [45]:
unique_values, counts = np.unique(y_train, return_counts=True)
ratio = dict(zip(unique_values, counts))
ratio

{0: 2240, 1: 332}

In [46]:
imbalance_ratio = ratio[0] / ratio[1]
imbalance_ratio

6.746987951807229

In [47]:
category_cols = list(X_train.select_dtypes(include='category').columns)
numeric_cols = list(X_train.select_dtypes(include=['number']).columns)
one_hot = OneHotEncoder(handle_unknown='ignore')
num_imputer = KNNImputer(n_neighbors=5)
len(category_cols), len(numeric_cols)

(4, 6289)

In [48]:
preprocess = ColumnTransformer([('numerical', num_imputer, numeric_cols), ('cat_encoder', one_hot, category_cols)], remainder='passthrough')
X_train_processed = preprocess.fit_transform(X_train)
X_train_processed.shape

(2572, 6304)

In [49]:
X_test_processed = preprocess.transform(X_test)
X_test_processed.shape

(1103, 6304)

In [50]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(sampling_strategy=0.5, random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [51]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling',
                 SMOTETomek(random_state=7, sampling_strategy=0.5)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=6.746987951807229, subsample=1,
                               tree_method='exact', use_label_encoder=False,
     

In [52]:
y_pred = pipeline.predict(X_test_processed)

In [53]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[908,  52],
       [ 80,  63]])

In [54]:
roc_auc_score(y_test, y_pred)

0.693196386946387

In [55]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93       960
           1       0.55      0.44      0.49       143

    accuracy                           0.88      1103
   macro avg       0.73      0.69      0.71      1103
weighted avg       0.87      0.88      0.87      1103



In [56]:
print(matthews_corrcoef(y_test, y_pred))

0.4247224706406981


In [57]:
importance = pipeline.named_steps['model'].feature_importances_
importance.shape, type(importance)

((6304,), numpy.ndarray)

In [58]:
category_label = list(preprocess.named_transformers_['cat_encoder'].get_feature_names(category_cols))
feature_label = numeric_cols + category_label
len(feature_label)

6304

In [59]:
pd.set_option('display.max_rows', 400)
eval_df = pd.DataFrame({'label': feature_label, 'importance_value': importance})
eval_df.sort_values(by='importance_value', ascending=False).head(100)

,label,importance_value
2032,creatinine__quantile__q_0.4,0.040011
4532,eosinophils__approximate_entropy__m_2__r_0.9,0.032294
1957,creatinine__median,0.030981
981,alkphos__variance_larger_than_standard_deviation,0.023078
2244,egfr__time_reversal_asymmetry_statistic__lag_2,0.013753
1841,"chloride__change_quantiles__f_agg_""""""""""""""""mean...",0.013141
4999,lymphocytes__median,0.012887
5503,mchc__quantile__q_0.6,0.012206
3386,platelet__approximate_entropy__m_2__r_0.3,0.010835
4113,alt__energy_ratio_by_chunks__num_segments_10__...,0.010218


## ESKD Prediction 2 years post-training

In [60]:
X = df_rm_intrain.drop(dropped_cols,axis=1).copy()
y = df_rm_intrain['cat3.5']
X.shape, y.shape

((3675, 6293), (3675,))

In [61]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3675 entries, 3916 to 23864
Columns: 6293 entries, urea__variance_larger_than_standard_deviation to age.init
dtypes: category(4), float64(6288), int64(1)
memory usage: 176.4 MB


In [62]:
df_rm_intrain['cat3.5'].value_counts()

non_eskd    3444
eskd         231
Name: cat3.5, dtype: int64

In [63]:
y_mapped = y.map({'non_eskd':'no', 'eskd':'yes'})
y_mapped

id
3916     no
3918     no
3921     no
3924     no
3930     no
         ..
23760    no
23771    no
23780    no
23823    no
23864    no
Name: cat3.5, Length: 3675, dtype: category
Categories (2, object): ['yes', 'no']

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, stratify=y_mapped, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2572, 6293), (1103, 6293), (2572,), (1103,))

In [65]:
lab_encode = LabelEncoder()
y_train = lab_encode.fit_transform(y_train)
y_test = lab_encode.transform(y_test)
y_train.shape, y_test.shape

((2572,), (1103,))

In [66]:
unique_values, counts = np.unique(y_train, return_counts=True)
ratio = dict(zip(unique_values, counts))
ratio

{0: 2410, 1: 162}

In [67]:
imbalance_ratio = ratio[0] / ratio[1]
imbalance_ratio

14.876543209876543

In [68]:
category_cols = list(X_train.select_dtypes(include='category').columns)
numeric_cols = list(X_train.select_dtypes(include=['int', 'float']).columns)
one_hot = OneHotEncoder(handle_unknown='ignore')
num_imputer = KNNImputer(n_neighbors=5)
len(category_cols), len(numeric_cols)

(4, 6289)

In [69]:
preprocess = ColumnTransformer([('num_imputing', num_imputer, numeric_cols), ('cat_encoder', one_hot, category_cols)], remainder='passthrough')
X_train_processed = preprocess.fit_transform(X_train)
X_train_processed.shape

(2572, 6304)

In [70]:
X_test_processed = preprocess.transform(X_test)

In [71]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [72]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling', SMOTETomek(random_state=7)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=14.876543209876543, subsample=1,
                               tree_method='exact', use_label_encoder=False,
                               validate_para

In [73]:
y_pred = pipeline.predict(X_test_processed)

In [74]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[998,  36],
       [ 49,  20]])

In [75]:
roc_auc_score(y_test, y_pred)

0.6275194124407816

In [76]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1034
           1       0.36      0.29      0.32        69

    accuracy                           0.92      1103
   macro avg       0.66      0.63      0.64      1103
weighted avg       0.92      0.92      0.92      1103



In [32]:
print(matthews_corrcoef(y_test, y_pred))

0.28034459959348784


In [33]:
importance = pipeline.named_steps['model'].feature_importances_
importance.shape, type(importance)

((5355,), numpy.ndarray)

In [34]:
category_label = list(preprocess.named_transformers_['cat_encoder'].get_feature_names(category_cols))
feature_label = numeric_cols + category_label
len(feature_label)

5355

In [35]:
pd.set_option('display.max_rows', 400)
eval_df = pd.DataFrame({'label': feature_label, 'importance_value': importance})
eval_df.sort_values(by='importance_value', ascending=False).head(100)

,label,importance_value
4033,eosinophils__large_standard_deviation__r_0.300...,0.070197
4605,"haemoglobin__fft_coefficient__attr_""""""""""""""""ima...",0.041579
3860,"egfr__change_quantiles__f_agg_""""""""""""""""mean""""""""...",0.032070
3517,creatinine__benford_correlation,0.030442
5080,mch__ratio_beyond_r_sigma__r_0.5,0.021785
4799,"lymphocytes__change_quantiles__f_agg_""""""""""""""""v...",0.021245
3572,creatinine__quantile__q_0.9,0.017708
3759,egfr__time_reversal_asymmetry_statistic__lag_2,0.014997
737,phosphate__quantile__q_0.9,0.013442
5021,"mch__change_quantiles__f_agg_""""""""""""""""mean""""""""""...",0.013026
